In [5]:
import numpy as np
import pandas as pd
import os
import re
import shutil

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

from collections import defaultdict
from glob import glob
import random

import cv2
from PIL import Image
matplotlib.use('Agg')   # type: ignore
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [6]:
# Define dataset directory
images_dir = "train-raw/arrays"

# Training parameters
img_size = (32, 32)
batch_size = 32
epochs = 50

# Dictionary to store accuracies
max_accuracies = {}

In [7]:
# Function to build a simple CNN model
def cnn_model(input_shape, num_classes):
	model = Sequential([
		Conv2D(32, (5,5), activation='relu', input_shape=input_shape),
		MaxPooling2D(2,2),
		Conv2D(64, (5,5), activation='relu'),
		MaxPooling2D(2,2),
		Flatten(),
		Dense(128, activation='relu'),
		Dense(num_classes, activation='softmax')  # Adjust output neurons based on classes
	])
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [8]:
# Dataset Preparation and Undersampling
def prepare_dataset(method_path):
	class_dirs = [d for d in os.listdir(method_path) if os.path.isdir(os.path.join(method_path, d))]
	class_to_images = defaultdict(list)
	for class_name in class_dirs:
		class_path = os.path.join(method_path, class_name)
		image_paths = glob(os.path.join(class_path, "*"))
		class_to_images[class_name].extend(image_paths)

	target_size = min(
		max(1000 if k in ['Euro-American', 'East Asian'] else len(v), len(v))
		for k, v in class_to_images.items()
	)
	balanced_image_paths = []
	balanced_labels = []
	for cls, imgs in class_to_images.items():
		sampled = random.sample(imgs, target_size) if len(imgs) > target_size else imgs
		balanced_image_paths.extend(sampled)
		balanced_labels.extend([cls] * len(sampled))

	combined = list(zip(balanced_image_paths, balanced_labels))
	random.shuffle(combined)
	balanced_image_paths, balanced_labels = zip(*combined)
	df = pd.DataFrame({'filename': balanced_image_paths, 'class': balanced_labels})
	return df

In [9]:
# Iterate through resolution folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(images_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		# Iterate through method folders inside resolution
		for method_folder in os.listdir(resolution_path):
			method_path = os.path.join(resolution_path, method_folder)

			if os.path.isdir(method_path):
				print(f"\nTraining {resolution_folder}/{method_folder}...")
    
				# Prepare dataset
				df = prepare_dataset(method_path)

				# Data Preprocessing
				datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

				# Use subset='training' and 'validation' for split
				train_data = datagen.flow_from_dataframe(
					dataframe=df,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='training'
				)
				val_data = datagen.flow_from_dataframe(
					dataframe=df,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='validation',
					shuffle=False
				)

				num_samples = len(df)
				num_classes = len(df['class'].unique())
				input_shape = (img_size[0], img_size[1], 3)

				# Initialize and train the model
				model = cnn_model(input_shape, num_classes)
				history = model.fit(
					train_data,
					validation_data=val_data,
					epochs=epochs,
					verbose=1
				)

				# Save the model
				model_path = os.path.join("models/standard", resolution_folder, method_folder)
				os.makedirs(model_path, exist_ok=True)
				model_name = f"{resolution_folder}_{method_folder}_mask_5_undersample.h5"
				model.save(os.path.join(model_path, model_name))
				print(f"Model saved to {model_path}")

				# Store final validation accuracy
				final_val_acc = max(history.history['val_accuracy'])
				key = f"{resolution_folder} - {method_folder}"
				max_accuracies[key] = final_val_acc
				print(f"Validation accuracy for {key}: {final_val_acc:.4f}")



Training 4px/Chargaff-Composante-Diversite...
Found 668 validated image filenames belonging to 5 classes.
Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 31ms/step - loss: 1.6197 - accuracy: 0.1976 - val_loss: 1.6015 - val_accuracy: 0.2575
Epoch 2/50
21/21 [==============================] - 0s 21ms/step - loss: 1.5839 - accuracy: 0.2814 - val_loss: 1.5592 - val_accuracy: 0.3114
Epoch 3/50
21/21 [==============================] - 1s 24ms/step - loss: 1.5409 - accuracy: 0.2889 - val_loss: 1.5151 - val_accuracy: 0.3892
Epoch 4/50
21/21 [==============================] - 1s 24ms/step - loss: 1.5010 - accuracy: 0.3234 - val_loss: 1.4872 - val_accuracy: 0.4072
Epoch 5/50
21/21 [==============================] - 0s 23ms/step - loss: 1.4733 - accuracy: 0.3428 - val_loss: 1.4641 - val_accuracy: 0.4132
Epoch 6/50
21/21 [==============================] - 1s 24ms/step - loss: 1.4587 - accuracy: 0.3533 - val_loss: 1.5224 - val_accu

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 34ms/step - loss: 1.6208 - accuracy: 0.2141 - val_loss: 1.6326 - val_accuracy: 0.1916
Epoch 2/50
21/21 [==============================] - 0s 23ms/step - loss: 1.6161 - accuracy: 0.1751 - val_loss: 1.6092 - val_accuracy: 0.1916
Epoch 3/50
21/21 [==============================] - 0s 23ms/step - loss: 1.6144 - accuracy: 0.1781 - val_loss: 1.6093 - val_accuracy: 0.2036
Epoch 4/50
21/21 [==============================] - 1s 25ms/step - loss: 1.6098 - accuracy: 0.1856 - val_loss: 1.6097 - val_accuracy: 0.2874
Epoch 5/50
21/21 [==============================] - 0s 23ms/step - loss: 1.6096 - accuracy: 0.2081 - val_loss: 1.6090 - val_accuracy: 0.2036
Epoch 6/50
21/21 [==============================] - 0s 19ms/step - loss: 1.6094 - accuracy: 0.1781 - val_loss: 1.6059 - val_accuracy: 0.2036
Epoch 7/50
21/21 [==============================] - 0s 19ms/step - loss: 1.6088 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 29ms/step - loss: 1.6490 - accuracy: 0.2066 - val_loss: 1.6107 - val_accuracy: 0.1677
Epoch 2/50
21/21 [==============================] - 0s 18ms/step - loss: 1.6020 - accuracy: 0.2485 - val_loss: 1.5706 - val_accuracy: 0.3114
Epoch 3/50
21/21 [==============================] - 0s 20ms/step - loss: 1.5613 - accuracy: 0.2725 - val_loss: 1.5052 - val_accuracy: 0.3473
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 1.5075 - accuracy: 0.3174 - val_loss: 1.4834 - val_accuracy: 0.3653
Epoch 5/50
21/21 [==============================] - 0s 19ms/step - loss: 1.4981 - accuracy: 0.3114 - val_loss: 1.4612 - val_accuracy: 0.3054
Epoch 6/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4927 - accuracy: 0.3219 - val_loss: 1.4466 - val_accuracy: 0.3653
Epoch 7/50
21/21 [==============================] - 0s 20ms/step - loss: 1.5017 - accuracy: 0.

...

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 27ms/step - loss: 1.5964 - accuracy: 0.2260 - val_loss: 1.5353 - val_accuracy: 0.3413
Epoch 2/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4507 - accuracy: 0.3802 - val_loss: 1.4517 - val_accuracy: 0.4671
Epoch 3/50
21/21 [==============================] - 0s 19ms/step - loss: 1.2976 - accuracy: 0.5045 - val_loss: 1.2973 - val_accuracy: 0.3892
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 1.0461 - accuracy: 0.5868 - val_loss: 0.9984 - val_accuracy: 0.6287
Epoch 5/50
21/21 [==============================] - 0s 20ms/step - loss: 0.7600 - accuracy: 0.7380 - val_loss: 0.8154 - val_accuracy: 0.6886
Epoch 6/50
21/21 [==============================] - 0s 19ms/step - loss: 0.6105 - accuracy: 0.8099 - val_loss: 0.7640 - val_accuracy: 0.7305
Epoch 7/50
21/21 [==============================] - 0s 19ms/step - loss: 0.5362 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 29ms/step - loss: 1.5852 - accuracy: 0.2605 - val_loss: 1.4576 - val_accuracy: 0.3832
Epoch 2/50
21/21 [==============================] - 0s 19ms/step - loss: 1.4513 - accuracy: 0.3802 - val_loss: 1.3783 - val_accuracy: 0.3952
Epoch 3/50
21/21 [==============================] - 0s 19ms/step - loss: 1.2127 - accuracy: 0.5000 - val_loss: 0.9989 - val_accuracy: 0.7365
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 0.8785 - accuracy: 0.7365 - val_loss: 0.7237 - val_accuracy: 0.7605
Epoch 5/50
21/21 [==============================] - 0s 22ms/step - loss: 0.6899 - accuracy: 0.7665 - val_loss: 0.7213 - val_accuracy: 0.7425
Epoch 6/50
21/21 [==============================] - 0s 21ms/step - loss: 0.5762 - accuracy: 0.8219 - val_loss: 0.5928 - val_accuracy: 0.8204
Epoch 7/50
21/21 [==============================] - 0s 22ms/step - loss: 0.4878 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Create positions for bars
num_bars = len(max_accuracies)
x_positions = np.arange(num_bars)

# Add extra space every 10 bars
for i in range(10, num_bars, 10):  
	x_positions[i:] += 1  # Shift everything after every 10th bar

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Validation Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/kfold/cnn_validation_accuracy_bar_mask_5-kfold_aug2.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\22334614.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))

# Organize data for plotting
data = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods}

# Define a color map for better distinction
colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors

# Plot the lines
plt.figure(figsize=(16, 10))
for i, (method, accuracies) in enumerate(data.items()):
	plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, label=method, color=colors(i))

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution and Method", fontsize=14)

# Move legend outside the plot for better clarity
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)

# Add a grid with transparency
plt.grid(True, linestyle="--", alpha=0.6)

# Improve layout to fit legend properly
plt.tight_layout()

# Show plot
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_validation_accuracy_mask_5_undersample.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\1036221758.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_11052\1036221758.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))

# Extract unique methods and split into two groups of 5
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))
methods_group1 = methods[:5]
methods_group2 = methods[5:]

# Organize data for plotting
data1 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group1}
data2 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group2}

# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True, sharey=True)

colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors

# Plot first group
for method, accuracies in data1.items():
	axs[0].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group1.index(method)))
axs[0].set_title("CNN Validation Accuracy (Group 1)")
axs[0].set_ylabel("Max Accuracy")
axs[0].legend(title="Method")
axs[0].grid(True, linestyle="--", alpha=0.6)

# Plot second group
for method, accuracies in data2.items():
	axs[1].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group2.index(method)))
axs[1].set_title("CNN Validation Accuracy (Group 2)")
axs[1].set_xlabel("Resolution")
axs[1].set_ylabel("Max Accuracy")
axs[1].legend(title="Method")
axs[1].grid(True, linestyle="--", alpha=0.6)

# Adjust layout
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_validation_accuracy_groups_mask_5_undersample.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\2309815400.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_11052\2309815400.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model
model_path = "models/standard/100px/Chargaff-Composante-Diversite/100px_Chargaff-Composante-Diversite_mask_5_undersample.h5"
model = load_model(model_path)
model.summary()

# Load the test data
test_data_dir = 'test-raw/arrays/100px/Chargaff-Composante-Diversite'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=img_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)
# Get the true labels
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Get the predicted labels
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix at 100px Resolution for Chargaff-Composante-Diversite Method')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_confusion_matrix_100px_mask_5_undersample.png")

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_80 (Conv2D)          (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d_80 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_81 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 max_pooling2d_81 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_40 (Flatten)        (None, 1600)              0         
                                                                 
 dense_80 (Dense)            (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\4153983215.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
# Define dataset directory
mosaics_dir = 'train-raw/mosaics'

# Training parameters
img_size = (20, 50)  # Resize images to 64x64
batch_size = 32
epochs = 50
 
# Dictionary to store max accuracies
max_accuracies_mos = {}

In [15]:
# Mosaic version (without cross-validation)
# Iterate through resolution folders (e.g., 4px, 16px)
for resolution_folder in sorted(os.listdir(mosaics_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(mosaics_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		print(f"\nTraining {resolution_folder}...")

		# Data Preprocessing
		datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
  
		# Prepare dataset
		new_df = prepare_dataset(resolution_path)

		# Create training and validation generators
		train_data = datagen.flow_from_dataframe(
			dataframe=new_df,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='training'
		)
		val_data = datagen.flow_from_dataframe(
			dataframe=new_df,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='validation',
			shuffle=False
		)

		num_samples = len(new_df)
		num_classes = len(new_df['class'].unique())
		input_shape = (img_size[0], img_size[1], 3)

		# Initialize and train model
		model = cnn_model(input_shape, num_classes)
		history = model.fit(
			train_data,
			validation_data=val_data,
			epochs=epochs,
			verbose=1
		)

		# Save the model
		model_path = os.path.join("models/standard-mosaic", resolution_folder)
		os.makedirs(model_path, exist_ok=True)
		model_name = f"{resolution_folder}_mosaic_mask_5_undersample.h5"
		model.save(os.path.join(model_path, model_name))
		print(f"Model saved to {model_path}")

		# Store final validation accuracy
		final_val_acc = max(history.history['val_accuracy'])
		max_accuracies_mos[resolution_folder] = final_val_acc
		print(f"Validation accuracy for {resolution_folder}: {final_val_acc:.4f}")


Training 4px...
Found 668 validated image filenames belonging to 5 classes.
Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 33ms/step - loss: 1.6319 - accuracy: 0.1931 - val_loss: 1.6073 - val_accuracy: 0.2275
Epoch 2/50
21/21 [==============================] - 0s 21ms/step - loss: 1.6131 - accuracy: 0.1901 - val_loss: 1.6075 - val_accuracy: 0.2275
Epoch 3/50
21/21 [==============================] - 0s 18ms/step - loss: 1.6067 - accuracy: 0.2171 - val_loss: 1.6025 - val_accuracy: 0.2395
Epoch 4/50
21/21 [==============================] - 0s 18ms/step - loss: 1.6014 - accuracy: 0.2650 - val_loss: 1.6036 - val_accuracy: 0.2036
Epoch 5/50
21/21 [==============================] - 0s 19ms/step - loss: 1.5929 - accuracy: 0.2395 - val_loss: 1.5916 - val_accuracy: 0.2216
Epoch 6/50
21/21 [==============================] - 0s 21ms/step - loss: 1.5827 - accuracy: 0.2365 - val_loss: 1.5710 - val_accuracy: 0.2515
Epoch 7/50
21/21 

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
21/21 [==============================] - 1s 28ms/step - loss: 1.6132 - accuracy: 0.2411 - val_loss: 1.6063 - val_accuracy: 0.1429
Epoch 2/50
21/21 [==============================] - 0s 19ms/step - loss: 1.5406 - accuracy: 0.3363 - val_loss: 1.5341 - val_accuracy: 0.2857
Epoch 3/50
21/21 [==============================] - 0s 19ms/step - loss: 1.3922 - accuracy: 0.4717 - val_loss: 1.4784 - val_accuracy: 0.3571
Epoch 4/50
21/21 [==============================] - 0s 18ms/step - loss: 1.3405 - accuracy: 0.4420 - val_loss: 1.3285 - val_accuracy: 0.4405
Epoch 5/50
21/21 [==============================] - 0s 19ms/step - loss: 1.1934 - accuracy: 0.5179 - val_loss: 1.2478 - val_accuracy: 0.4821
Epoch 6/50
21/21 [==============================] - 0s 20ms/step - loss: 1.1066 - accuracy: 0.5670 - val_loss: 1.2386 - val_accuracy: 0.5179
Epoch 7/50
21/21 [==============================] - 0s 20ms/step - loss: 0.9909 - accuracy: 0.6295 - val_loss: 1.1004 - val_accuracy: 0.5893
Epoch 8/50
21

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 672 validated image filenames belonging to 5 classes.
Found 168 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 35ms/step - loss: 1.6013 - accuracy: 0.2262 - val_loss: 1.5461 - val_accuracy: 0.4583
Epoch 2/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4924 - accuracy: 0.3914 - val_loss: 1.3997 - val_accuracy: 0.4702
Epoch 3/50
21/21 [==============================] - 0s 19ms/step - loss: 1.3013 - accuracy: 0.4271 - val_loss: 1.1979 - val_accuracy: 0.4524
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 1.1366 - accuracy: 0.5223 - val_loss: 1.2129 - val_accuracy: 0.4881
Epoch 5/50
21/21 [==============================] - 0s 18ms/step - loss: 1.0328 - accuracy: 0.5789 - val_loss: 1.0957 - val_accuracy: 0.4821
Epoch 6/50
21/21 [==============================] - 0s 19ms/step - loss: 0.9448 - accuracy: 0.6116 - val_loss: 1.0250 - val_accuracy: 0.5714
Epoch 7/50
21/21 [================

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 168 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 37ms/step - loss: 1.6101 - accuracy: 0.2783 - val_loss: 1.5911 - val_accuracy: 0.1905
Epoch 2/50
21/21 [==============================] - 0s 20ms/step - loss: 1.5436 - accuracy: 0.3021 - val_loss: 1.6038 - val_accuracy: 0.2202
Epoch 3/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4947 - accuracy: 0.3438 - val_loss: 1.4493 - val_accuracy: 0.4524
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4149 - accuracy: 0.3958 - val_loss: 1.3309 - val_accuracy: 0.5893
Epoch 5/50
21/21 [==============================] - 0s 20ms/step - loss: 1.2453 - accuracy: 0.5818 - val_loss: 1.1852 - val_accuracy: 0.6071
Epoch 6/50
21/21 [==============================] - 0s 21ms/step - loss: 1.0229 - accuracy: 0.6429 - val_loss: 0.9321 - val_accuracy: 0.6667
Epoch 7/50
21/21 [==============================] - 0s 22ms/step - loss: 0.8194 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 167 validated image filenames belonging to 5 classes.
Epoch 1/50
21/21 [==============================] - 1s 34ms/step - loss: 1.6083 - accuracy: 0.2440 - val_loss: 1.5590 - val_accuracy: 0.3174
Epoch 2/50
21/21 [==============================] - 0s 20ms/step - loss: 1.4741 - accuracy: 0.3847 - val_loss: 1.3611 - val_accuracy: 0.5030
Epoch 3/50
21/21 [==============================] - 0s 19ms/step - loss: 1.3405 - accuracy: 0.4416 - val_loss: 1.3097 - val_accuracy: 0.5449
Epoch 4/50
21/21 [==============================] - 0s 20ms/step - loss: 1.1932 - accuracy: 0.5509 - val_loss: 1.0532 - val_accuracy: 0.7126
Epoch 5/50
21/21 [==============================] - 0s 21ms/step - loss: 0.9540 - accuracy: 0.6871 - val_loss: 0.8590 - val_accuracy: 0.7904
Epoch 6/50
21/21 [==============================] - 0s 20ms/step - loss: 0.7712 - accuracy: 0.7784 - val_loss: 0.6968 - val_accuracy: 0.8204
Epoch 7/50
21/21 [==============================] - 0s 20ms/step - loss: 0.6465 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# Create positions for bars
num_bars = len(max_accuracies_mos)
x_positions = np.arange(num_bars)

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies_mos.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies_mos.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Max Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/standard_mosaic/cnn_validation_accuracy_mosaics_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\2427635834.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
# Extract and sort resolutions numerically
resolutions = sorted(max_accuracies_mos.keys(), key=lambda x: int(x.replace("px", "")))

# Get max accuracy values in the correct order
accuracies = [max_accuracies_mos[res] for res in resolutions]

# Plot the accuracy per resolution
plt.figure(figsize=(12, 6))
plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, color="blue", label="Accuracy")

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution", fontsize=14)

# Add data points on the plot
for i, acc in enumerate(accuracies):
    plt.text(resolutions[i], acc, f"{acc:.2f}", fontsize=10, ha="right")

# Grid and legend
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(loc="lower right")

# Save and show plot
plt.tight_layout()
plt.show()
plt.savefig("../../imgs/graphs/standard-mosaic/cnn_validation_accuracy_mosaics_line_mask_5_undersample.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\1984822632.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model for the new version
new_model_path = "models/standard-mosaic/100px/100px_mosaic_mask_5_undersample.h5"
new_model = load_model(new_model_path)
new_model.summary()

# Load the test data for the new version
new_test_data_dir = 'test-raw/mosaics/100px'

new_test_datagen = ImageDataGenerator(rescale=1./255)
new_test_generator = new_test_datagen.flow_from_directory(
	new_test_data_dir,
	target_size=(img_size[0], img_size[1]),
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)

# Get the true labels for the new version
new_true_labels = new_test_generator.classes
new_class_labels = list(new_test_generator.class_indices.keys())

# Get the predicted labels for the new version
new_predictions = new_model.predict(new_test_generator, steps=len(new_test_generator), verbose=1)
new_predicted_labels = np.argmax(new_predictions, axis=1)

# Generate the confusion matrix for the new version
new_cm = confusion_matrix(new_true_labels, new_predicted_labels)

# Plot the confusion matrix for the new version
plt.figure(figsize=(10, 8))
sns.heatmap(new_cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_class_labels, yticklabels=new_class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for Mosaic images at 100px')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard-mosaic/cnn_confusion_matrix_mosaics_100px_mask_5_undersample.png")

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_108 (Conv2D)         (None, 16, 46, 32)        2432      
                                                                 
 max_pooling2d_108 (MaxPool  (None, 8, 23, 32)         0         
 ing2D)                                                          
                                                                 
 conv2d_109 (Conv2D)         (None, 4, 19, 64)         51264     
                                                                 
 max_pooling2d_109 (MaxPool  (None, 2, 9, 64)          0         
 ing2D)                                                          
                                                                 
 flatten_54 (Flatten)        (None, 1152)              0         
                                                                 
 dense_108 (Dense)           (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_11052\456015347.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
